In [ ]:
%pip install kaggle kagglehub plotly numpy pandas pillow matplotlib ipywidgets

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("airbusgeo/airbus-aircrafts-sample-dataset")

print("Path to dataset files:", path)

In [ ]:
import numpy as np
import pandas as pd
import ast
import PIL
from pathlib import Path
import random

import matplotlib
%matplotlib inline

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()
!yolo checks

In [ ]:
!pip install albumentations==1.0.3
display.clear_output()
!pip show albumentations

In [ ]:
DATA_DIR = Path(path)
img_list = list(DATA_DIR.glob('images/*.jpg'))
pickone = random.choice(img_list)
display.Image(pickone)

In [ ]:
print(f"Found {len(img_list)} images files in {DATA_DIR}")

img = PIL.Image.open(pickone)
IMAGE_HEIGHT, IMAGE_WIDTH = img.size
num_channels = len(img.mode)
print("Image size: {}".format((IMAGE_HEIGHT, IMAGE_WIDTH)))
print("Num channels: {}".format(num_channels))

In [ ]:
df = pd.read_csv(DATA_DIR / 'annotations.csv')
# convert a string record into a valid python object
def f(x):
    return ast.literal_eval(x.rstrip('\r\n'))

df = pd.read_csv(DATA_DIR / "annotations.csv",
                converters={'geometry': f})
df.head(10)

In [ ]:
def getBounds(geometry):
    try:
        arr = np.array(geometry).T
        xmin = np.min(arr[0])
        ymin = np.min(arr[1])
        xmax = np.max(arr[0])
        ymax = np.max(arr[1])
        return (xmin, ymin, xmax, ymax)
    except:
        return np.nan

def getWidth(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(xmax - xmin)
    except:
        return np.nan

def getHeight(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(ymax - ymin)
    except:
        return np.nan

# Create bounds, width and height
df.loc[:,'bounds'] = df.loc[:,'geometry'].apply(getBounds)
df.loc[:,'width'] = df.loc[:,'bounds'].apply(getWidth)
df.loc[:,'height'] = df.loc[:,'bounds'].apply(getHeight)
df.head(10)

In [ ]:
# create a list of images used for validation
fold = 1
num_fold = 5
index = df['image_id'].unique()
val_indexes = index[len(index)*fold//num_fold:len(index)*(fold+1)//num_fold]
print(val_indexes)

In [ ]:
##%%
import os
import tqdm.notebook

# Create 512x512 tiles with 64 pix overlap in /kaggle/working
TILE_WIDTH = 512
TILE_HEIGHT = 512
TILE_OVERLAP = 64
TRUNCATED_PERCENT = 0.3
_overwriteFiles = True

TILES_DIR = {'train': Path('kaggle/working/train/images/'),
             'val': Path('kaggle/working/val/images/')}
for _, folder in TILES_DIR.items():
    if not os.path.isdir(folder):
        os.makedirs(folder)

LABELS_DIR = {'train': Path('kaggle/working/train/labels/'),
              'val': Path('kaggle/working/val/labels/')}
for _, folder in LABELS_DIR.items():
    if not os.path.isdir(folder):
        os.makedirs(folder)

# Save one line in .txt file for each tag found inside the tile
def tag_is_inside_tile(bounds, x_start, y_start, width, height, truncated_percent):
    x_min, y_min, x_max, y_max = bounds
    x_min, y_min, x_max, y_max = x_min - x_start, y_min - y_start, x_max - x_start, y_max - y_start

    if (x_min > width) or (x_max < 0.0) or (y_min > height) or (y_max < 0.0):
        return None

    x_max_trunc = min(x_max, width)
    x_min_trunc = max(x_min, 0)
    if (x_max_trunc - x_min_trunc) / (x_max - x_min) < truncated_percent:
        return None

    y_max_trunc = min(y_max, width)
    y_min_trunc = max(y_min, 0)
    if (y_max_trunc - y_min_trunc) / (y_max - y_min) < truncated_percent:
        return None

    x_center = (x_min_trunc + x_max_trunc) / 2.0 / width
    y_center = (y_min_trunc + y_max_trunc) / 2.0 / height
    x_extend = (x_max_trunc - x_min_trunc) / width
    y_extend = (y_max_trunc - y_min_trunc) / height

    return (0, x_center, y_center, x_extend, y_extend)

for img_path in tqdm.notebook.tqdm(img_list):
    # Open image and related data
    pil_img = PIL.Image.open(img_path, mode='r')
    np_img = np.array(pil_img, dtype=np.uint8)

    # Get annotations for image
    img_labels = df[df["image_id"] == img_path.name]
    #print(img_labels)

    # Count number of sections to make
    X_TILES = (IMAGE_WIDTH + TILE_WIDTH + TILE_OVERLAP - 1) // TILE_WIDTH
    Y_TILES = (IMAGE_HEIGHT + TILE_HEIGHT + TILE_OVERLAP - 1) // TILE_HEIGHT

    # Cut each tile
    for x in range(X_TILES):
        for y in range(Y_TILES):

            x_end = min((x + 1) * TILE_WIDTH - TILE_OVERLAP * (x != 0), IMAGE_WIDTH)
            x_start = x_end - TILE_WIDTH
            y_end = min((y + 1) * TILE_HEIGHT - TILE_OVERLAP * (y != 0), IMAGE_HEIGHT)
            y_start = y_end - TILE_HEIGHT
            #print(x_start, y_start)

            folder = 'val' if img_path.name in val_indexes else 'train'
            save_tile_path = TILES_DIR[folder].joinpath(img_path.stem + "_" + str(x_start) + "_" + str(y_start) + ".jpg")
            save_label_path = LABELS_DIR[folder].joinpath(img_path.stem + "_" + str(x_start) + "_" + str(y_start) + ".txt")

            # Save if file doesn't exit
            if _overwriteFiles or not os.path.isfile(save_tile_path):
                cut_tile = np.zeros(shape=(TILE_WIDTH, TILE_HEIGHT, 3), dtype=np.uint8)
                cut_tile[0:TILE_HEIGHT, 0:TILE_WIDTH, :] = np_img[y_start:y_end, x_start:x_end, :]
                cut_tile_img = PIL.Image.fromarray(cut_tile)
                cut_tile_img.save(save_tile_path)

            found_tags = [
                tag_is_inside_tile(bounds, x_start, y_start, TILE_WIDTH, TILE_HEIGHT, TRUNCATED_PERCENT)
                for i, bounds in enumerate(img_labels['bounds'])]
            found_tags = [el for el in found_tags if el is not None]

            # save labels
            with open(save_label_path, 'w+') as f:
                for tags in found_tags:
                    f.write(' '.join(str(x) for x in tags) + '\n')

In [ ]:
CONFIG = """
# train and val datasets (image directory or *.txt file with image paths)
train: train/
val: val/

# number of classes
nc: 1

# class names
names: ['Aircraft']
"""

with open("kaggle/working/data.yaml", "w") as f:
    f.write(CONFIG)

In [ ]:
HOME = "kaggle/working/"
!yolo task=detect mode=train model=yolov8s.pt data={HOME}/data.yaml epochs=10 imgsz=512

In [ ]:
display.Image(filename='runs/detect/train/BoxF1_curve.png', width=600)

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_csv("runs/detect/train/results.csv")
fig = px.line(df, x='epoch', y='metrics/mAP50(B)', title='mAP50')
fig.show()

In [ ]:
display.Image(filename=f'runs/detect/train/val_batch0_pred.jpg', width=1000)

In [ ]:
!yolo task=detect mode=val model=runs/detect/train/weights/best.pt data={HOME}/data.yaml

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load your trained model
model_path = "runs/detect/train3/weights/best.pt"  # Update this path if needed
model = YOLO(model_path)

print(f"(Model loaded)")

In [ ]:
# Path to your test images folder
test_images_path = ".cache/kagglehub/datasets/airbusgeo/airbus-aircrafts-sample-dataset/versions/3/extras"  # Change this to your test images folder

# Get all images in the test folder
test_images = [str(p) for p in Path(test_images_path).glob("*.jpg")]
print(f"Found {len(test_images)} test images")

In [ ]:
# Set confidence threshold
conf_threshold = 0.25

# Run inference on all test images
results = model(test_images, conf=conf_threshold)

print(f"Inference completed for {len(results)} images")

In [ ]:
# Function to visualize an image with bounding boxes
def visualize_with_boxes(image_path, result, save_path=None):
    """
    Visualize image with predicted bounding boxes

    Args:
        image_path: Path to the image
        result: YOLO result object for the image
        save_path: Path to save the visualization (optional)
    """
    # Load the image
    img = Image.open(image_path)
    img_np = np.array(img)

    # Create figure and axis
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(img_np)

    # Get the predictions (boxes, scores, and class indices)
    boxes = result.boxes.xyxy.cpu().numpy()
    scores = result.boxes.conf.cpu().numpy()
    class_ids = result.boxes.cls.cpu().numpy().astype(int)
    class_names = [result.names[class_id] for class_id in class_ids]

    # Loop through the detections and add bounding boxes
    for box, score, class_name in zip(boxes, scores, class_names):
        x1, y1, x2, y2 = box
        width = x2 - x1
        height = y2 - y1

        # Create a rectangle patch
        rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

        # Add label with class name and confidence score
        plt.text(x1, y1-10, f"{class_name}: {score:.2f}",
                 bbox=dict(facecolor='red', alpha=0.5),
                 color='white', fontsize=12)

    # Set title with the filename
    plt.title(f"Predictions for {os.path.basename(image_path)}")

    # Remove axis ticks
    plt.axis('off')

    # Save if a save path is provided
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
        print(f"Saved visualization to {save_path}")

    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
# Create output directory for visualizations
output_dir = "visualization_results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Visualize each result
for i, (img_path, result) in enumerate(zip(test_images, results)):
    print(f"Visualizing image {i+1}/{len(results)}: {os.path.basename(img_path)}")

    # Path to save the visualization
    save_path = os.path.join(output_dir, f"viz_{os.path.basename(img_path)}")

    # Visualize and save
    visualize_with_boxes(img_path, result, save_path)